In [1]:
## Combine all the correlation stats to one file

# Libraries

In [2]:
import anndata as ad
import numpy as np
import pandas as pd
import scanpy as sc
from matplotlib import pyplot as plt
from scipy.sparse import issparse

In [3]:
import os

In [4]:
import scipy.stats as stats
import numpy as np

In [5]:
from tqdm.auto import tqdm, trange

/home/icb/corinna.losert/miniconda3/envs/scgrn_R_4_1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Parameters

In [6]:
### Define dataset and cell-type for which to prepare

In [7]:
dataset = 'oneK1K'

In [8]:
cell_type = 'CD8_T'

In [9]:
### Path to the correlation files

In [10]:
data_path ='../data/current/coeqtl_mapping/co_qtls_sceqtlgen/correlations/'

## co_qtls_sceqtlgen can be replaced by co_qtls_decision_tree
## co_qtls_sceqtlgen: contains all for initial mapping run
## co_qtls_decision_tree: files for final decision tree filtered run

In [ ]:
### Path to the calculated expression summary stats

In [11]:
data_path_summary_stats = '/lustre/groups/epigenereg01/workspace/projects/grn_dev_groningen/summary_stats/'

## Files: Sample_gene_statistic  (info about genes on sample level)
#         Sample_summary (information about samples)
#         Sample_cell statistic

In [ ]:
### Path to save calculated summaries to

In [12]:
result_path = '../results/current/F6/'

In [13]:
result_path_analysis = "../data/current/coeqtl_mapping/co_qtls_sceqtlgen/analysis_"  + dataset + "/" + cell_type + "/"

# Functions

In [14]:
def get_r_thres(n, p_thres=0.975):
    t = stats.t.ppf(p_thres, n)
    return np.sqrt(1 / (((n - 2) / t ** 2) + 1))

# Data

## Correlation summary stats

In [ ]:
### Load from first chromosome to append the others

In [15]:
chromosome = 'chr-1'

In [16]:
result_path_analysis

'../data/current/coeqtl_mapping/co_qtls_decision_tree/analysis_oneK1K/CD8_T/'

In [17]:
correlation_stats =pd.read_csv(result_path_analysis +  'F6_Correlation_Summary_Stats_' + chromosome + '.csv')

In [18]:
correlation_stats.columns = ['Test', 'gene_pair', 'mean_correlation', 'mean_abs_correlation',
       'var_correlation', 'max_correlation', 'n_NA', 'n_not_NA']

In [19]:
#correlation_stats

In [20]:
len(pd.unique(correlation_stats['gene_pair']))

9249741

In [21]:
correlation_stats.to_csv(result_path_analysis + 'F6_Correlation_Summary_Stats_all.csv')  # Generate initial file

In [22]:
### Add the other files ( remove duplicates and save)

In [23]:
for chrom in trange(1, 23, desc="Chrom"):
#for chrom in [3,4,5,6,7,8,9,10,11,12,13,14,15,16,18,21,22]:
    chromosome = "chr-" + str(chrom)
    all_stats = pd.read_csv(result_path_analysis + 'F6_Correlation_Summary_Stats_all.csv')
    stats =pd.read_csv(result_path_analysis +  'F6_Correlation_Summary_Stats_' + chromosome + '.csv')
    stats.columns = ['Test', 'gene_pair', 'mean_correlation', 'mean_abs_correlation',
       'var_correlation', 'max_correlation', 'n_NA', 'n_not_NA']
    
    stats = stats[stats['gene_pair'].isin(all_stats['gene_pair']) == False]  # filter out already included gene-pairs
    all_stats = all_stats._append(stats)   # append all in one file
    
    all_stats.to_csv(result_path_analysis + 'F6_Correlation_Summary_Stats_all.csv')  # Save the appended file
    
    
    

Chrom: 100%|██████████| 22/22 [3:38:01<00:00, 594.61s/it]  
